In [1]:

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import math
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.preparation import fetch_dataset
from src.processing import split_train_test_val, dataset_to_array, dataset_to_generator, make_tf_dataset
from src.modules.visualizer import do_heatmap
import numpy as np

In [2]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.layers import LSTM

In [3]:
# This is the size of the window that is fed into the DNN
window_size = 15
# The number of the features present in the dataset
num_of_features = 42
# Number of distinct labels in the output
label_length = 4
# Hyperparameter that defines the number of samples to work through
# before updating the internal model parameters.
batch_size = 25
# Epochs
ep = 100

In [4]:
# Create a model
model = Sequential()
model.add(LSTM(512, return_sequences=False, input_shape=(window_size, num_of_features)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(label_length, activation='softmax'))
model.compile(loss='binary_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy',])


In [5]:
from src.processing import make_tf_dataset, split_train_test

dataset = fetch_dataset('dataset_all_slim.pkl')

train, test, val = split_train_test_val(dataset)

data_train = make_tf_dataset(train, window_size, num_of_features, label_length ).batch(batch_size)
data_test = make_tf_dataset(test ,window_size, num_of_features, label_length).batch(batch_size)

model.fit(data_train, epochs=ep)


W0730 13:42:36.229768 4659004864 deprecation.py:323] From /Users/noresources/anaconda3/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:505: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


Epoch 1/100


W0730 13:42:37.029266 4659004864 deprecation.py:323] From /Users/noresources/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2386/2386 [==============================] - 736s 309ms/step - loss: 0.2677 - accuracy: 0.9039
Epoch 2/100
2386/2386 [==============================] - 654s 274ms/step - loss: 0.1687 - accuracy: 0.9384
Epoch 3/100
2386/2386 [==============================] - 725s 304ms/step - loss: 0.1437 - accuracy: 0.9466
Epoch 4/100
2386/2386 [==============================] - 751s 315ms/step - loss: 0.1215 - accuracy: 0.9543
Epoch 5/100
2386/2386 [==============================] - 657s 275ms/step - loss: 0.1157 - accuracy: 0.9563
Epoch 6/100
2386/2386 [==============================] - 658s 276ms/step - loss: 0.1049 - accuracy: 0.9610
Epoch 7/100
2386/2386 [==============================] - 659s 276ms/step - loss: 0.0964 - accuracy: 0.9641
Epoch 8/100
2386/2386 [==============================] - 654s 274ms/step - loss: 0.0894 - accuracy: 0.9665
Epoch 9/100
2386/2386 [==============================] - 660s 277ms/step - loss: 0.0830 - accuracy: 0.9692
Epoch 10/100
2386/2386 [=========================

2386/2386 [==============================] - 752s 315ms/step - loss: 0.0117 - accuracy: 0.9961
Epoch 78/100
2386/2386 [==============================] - 715s 300ms/step - loss: 0.0139 - accuracy: 0.9954
Epoch 79/100
2386/2386 [==============================] - 648s 271ms/step - loss: 0.0133 - accuracy: 0.9958
Epoch 80/100
2386/2386 [==============================] - 647s 271ms/step - loss: 0.0141 - accuracy: 0.9957
Epoch 81/100
2386/2386 [==============================] - 645s 270ms/step - loss: 0.0150 - accuracy: 0.9953
Epoch 82/100
2386/2386 [==============================] - 644s 270ms/step - loss: 0.0143 - accuracy: 0.9953
Epoch 83/100
2386/2386 [==============================] - 645s 270ms/step - loss: 0.0129 - accuracy: 0.9960
Epoch 84/100
2386/2386 [==============================] - 645s 270ms/step - loss: 0.0122 - accuracy: 0.9959
Epoch 85/100
2386/2386 [==============================] - 647s 271ms/step - loss: 0.0131 - accuracy: 0.9957
Epoch 86/100
2386/2386 [=================

In [7]:
data_val = make_tf_dataset(val ,window_size, num_of_features, label_length).batch(batch_size)

results = model.evaluate(data_val)

    493/Unknown - 35s 72ms/step - loss: 0.2483 - accuracy: 0.9645

In [15]:
from tensorflow.compat.v1.math import confusion_matrix

def list_to_num(el):
    if el == [1,0,0,0]:
        return 0
    elif el == [0,1,0,0]:
        return 1
    elif el == [0,0,1,0]:
        return 2
    elif el == [0,0,0,1]:
        return 3
    else:
        print('Error',el)
        
def test_model(md, dt):
    print('\n')
    data_test = make_tf_dataset(dt, window_size, num_of_features, label_length ).batch(batch_size)
    results = md.evaluate(data_test)
    
    ph = []
    predictions = md.predict(data_test)
    for pred in predictions:
        i = list(pred).index(max(list(pred)))
        ph.append(i)
    # Ground truth
    fm , labels = dataset_to_array(dt)
    # Padding
    ph = ph + [3 for i in range (0, len(labels) - len(ph))]
    
    # -----------------
    labels = [list_to_num(list(x)) for x in labels]
    # CM
    cm = confusion_matrix(labels, ph)
    print('\n',cm)
    
test_model(model, val)



    493/Unknown - 50s 101ms/step - loss: 0.2483 - accuracy: 0.9645
 tf.Tensor(
[[3244   45   11  151]
 [  30 5463    0  244]
 [  12   17  573  168]
 [ 103   76   35 2157]], shape=(4, 4), dtype=int32)


In [10]:
dataset = fetch_dataset('dataset_a1_f1_slim.pkl')
data_train = make_tf_dataset(dataset, window_size, num_of_features, label_length ).batch(batch_size)



model.fit(data_train, epochs=ep)


W0728 19:44:36.695498 4579685824 deprecation.py:323] From /Users/noresources/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
    159/Unknown - 7s 44ms/step - loss: 0.2982 - accuracy: 0.8811 6s 51

KeyboardInterrupt: 

In [ ]:
model.save('model_a1_f1_{0}.h5'.format(ep))  # creates a HDF5 file 'my_model.h5'


In [ ]:
dataset = fetch_dataset('dataset_g1_slim.pkl')
data_test = make_tf_dataset(dataset, window_size, num_of_features, label_length ).batch(batch_size)
results = model.evaluate(data_test)

In [ ]:
dataset = fetch_dataset('dataset_f1_g1_slim.pkl')
data_train = make_tf_dataset(dataset, window_size, num_of_features, label_length ).batch(batch_size)


model.save('model_f1_g1_{0}.h5'.format(ep))  # creates a HDF5 file 'my_model.h5'

model.fit(data_train, epochs=ep)



In [ ]:
dataset = fetch_dataset('dataset_f1_slim.pkl')
data_test = make_tf_dataset(dataset, window_size, num_of_features, label_length ).batch(batch_size)
results = model.evaluate(data_test)

In [ ]:
dataset = 